# Creating a custom Estimator from an existing Keras model

In [2]:
import tensorflow as tf
import numpy as np

np.random.seed(1)

## Create the data
x = np.random.uniform(low=-1, high=1, size=(200,2))
y = np.ones(len(x))
y[x[:,0]*x[:,1]<0] = 0

x_train = x[:100,:]
y_train = y[:100]
x_valid = x[100:,:]
y_valid = y[100:]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,), name='input-features'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2022-02-02 14:50:08.218875: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-02-02 14:50:08.289695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 14:50:08.290063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2022-02-02 14:50:08.290121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 14:50:08.290674: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000

In [3]:
## Step 1: Define the input functions
def train_input_fn(x_train, y_train, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input-features':x_train}, y_train.reshape(-1,1)))
    
    # shuffle, repeat, and batch the examples.
    return dataset.shuffle(100).repeat().batch(batch_size)

In [4]:
def eval_input_fn(x_test, y_test=None, batch_size=8):
    if y_test is None:
        dataset = tf.data.Dataset.from_tensor_slices(
            {'input-features':x_test})
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({'input-features':x_train}, y_train.reshape(-1,1)))
        
    # shuffle, repeat, and batch the examples.
    return dataset.batch(batch_size)

In [5]:
## Step 2: Define the feature columns
features = [
    tf.feature_column.numeric_column(
        key='input-features',shape=(2,))
]

In [7]:
## Step 3: Convert the model to an Estimator
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

my_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir='models/estimator-for-XOR/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Using config: {'_model_dir': 'models/estimator-for-XOR/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff86c792050>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_

2022-02-02 14:58:47.326109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 14:58:47.326380: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2022-02-02 14:58:47.326451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 14:58:47.326833: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:02:00.0
2022-02-02 14:58:47.326868: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libc

In [8]:
## Step 4: Use the estimator
num_epochs = 200
batch_size = 2
steps_per_epoch = np.ceil(len(x_train)/batch_size)

my_estimator.train(
    input_fn=lambda: train_input_fn(x_train, y_train, batch_size),
    steps=num_epochs*steps_per_epoch)

my_estimator.evaluate(
    input_fn=lambda: eval_input_fn(x_valid, y_valid, batch_size))

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='models/estimator-for-XOR/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: models/estimator-for-XOR/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 8 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into models/estimator-for-XOR/model.ckpt.


2022-02-02 15:02:35.976080: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 15:02:35.976351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2022-02-02 15:02:35.976451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 15:02:35.976885: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:02:00.0
2022-02-02 15:02:35.976925: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libc

INFO:tensorflow:loss = 0.6724751, step = 0


2022-02-02 15:02:36.230882: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0


INFO:tensorflow:global_step/sec: 767.42
INFO:tensorflow:loss = 0.68331754, step = 100 (0.131 sec)
INFO:tensorflow:global_step/sec: 1156.58
INFO:tensorflow:loss = 0.6987574, step = 200 (0.086 sec)
INFO:tensorflow:global_step/sec: 1016.9
INFO:tensorflow:loss = 0.68493104, step = 300 (0.098 sec)
INFO:tensorflow:global_step/sec: 1072.81
INFO:tensorflow:loss = 0.69420564, step = 400 (0.093 sec)
INFO:tensorflow:global_step/sec: 931.211
INFO:tensorflow:loss = 0.68350846, step = 500 (0.108 sec)
INFO:tensorflow:global_step/sec: 649.188
INFO:tensorflow:loss = 0.64716065, step = 600 (0.154 sec)
INFO:tensorflow:global_step/sec: 657.209
INFO:tensorflow:loss = 0.68454957, step = 700 (0.152 sec)
INFO:tensorflow:global_step/sec: 695.78
INFO:tensorflow:loss = 0.65864456, step = 800 (0.144 sec)
INFO:tensorflow:global_step/sec: 1071.93
INFO:tensorflow:loss = 0.6956168, step = 900 (0.093 sec)
INFO:tensorflow:global_step/sec: 1078.41
INFO:tensorflow:loss = 0.6891842, step = 1000 (0.093 sec)
INFO:tensorflow

INFO:tensorflow:global_step/sec: 1204.59
INFO:tensorflow:loss = 0.011179384, step = 8300 (0.083 sec)
INFO:tensorflow:global_step/sec: 1243.95
INFO:tensorflow:loss = 0.0023969987, step = 8400 (0.080 sec)
INFO:tensorflow:global_step/sec: 1248.38
INFO:tensorflow:loss = 0.27307758, step = 8500 (0.080 sec)
INFO:tensorflow:global_step/sec: 1302.46
INFO:tensorflow:loss = 0.0002207645, step = 8600 (0.077 sec)
INFO:tensorflow:global_step/sec: 1248.72
INFO:tensorflow:loss = 0.013365648, step = 8700 (0.080 sec)
INFO:tensorflow:global_step/sec: 1245.46
INFO:tensorflow:loss = 0.004968996, step = 8800 (0.080 sec)
INFO:tensorflow:global_step/sec: 1210.97
INFO:tensorflow:loss = 0.0056930627, step = 8900 (0.083 sec)
INFO:tensorflow:global_step/sec: 1284.54
INFO:tensorflow:loss = 0.32427976, step = 9000 (0.078 sec)
INFO:tensorflow:global_step/sec: 1264.11
INFO:tensorflow:loss = 0.009536342, step = 9100 (0.079 sec)
INFO:tensorflow:global_step/sec: 1303.6
INFO:tensorflow:loss = 0.47319803, step = 9200 (0.

2022-02-02 15:02:45.496978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 15:02:45.497242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2022-02-02 15:02:45.497307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 15:02:45.497662: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:02:00.0
2022-02-02 15:02:45.497689: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libc

{'binary_accuracy': 0.96, 'loss': 0.10105645, 'global_step': 10000}